### Imports


In [1]:
import mat4py as mpy
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.colors import sample_colorscale

import pickle

In [2]:
# Aachen load

data_loader = mpy.loadmat("data/Aachen/raw/Degradation_Prediction_Dataset_ISEA.mat")
df = pd.DataFrame.from_dict(data_loader["TDS"])

df

,Cell,Sample,History_Cycle,History,Target_Cycle_Expanded,Target_expanded,Target_Cycle,Target
0,1,1,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.834012342776987, 1.8321873038112761, 1.8303...","[105, 110, 115, 120, 125, 130, 135, 140, 145, ...","[1.7973466099593307, 1.7957194104838883, 1.794...","[105, 150, 195, 240, 285, 330, 375, 420, 465, ...","[1.7973466099593307, 1.7832393947563925, 1.770..."
1,1,2,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.834012342776987, 1.8321873038112761, 1.8303...","[110, 115, 120, 125, 130, 135, 140, 145, 150, ...","[1.7957194104838883, 1.794106179214274, 1.7925...","[110, 155, 200, 245, 290, 335, 380, 425, 470, ...","[1.7957194104838883, 1.7817527911332156, 1.769..."
2,1,3,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.834012342776987, 1.8321873038112761, 1.8303...","[115, 120, 125, 130, 135, 140, 145, 150, 155, ...","[1.794106179214274, 1.7925073295393719, 1.7909...","[115, 160, 205, 250, 295, 340, 385, 430, 475, ...","[1.794106179214274, 1.7802836132202329, 1.7676..."
3,1,4,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.834012342776987, 1.8321873038112761, 1.8303...","[120, 125, 130, 135, 140, 145, 150, 155, 160, ...","[1.7925073295393719, 1.7909232748480652, 1.789...","[120, 165, 210, 255, 300, 345, 390, 435, 480, ...","[1.7925073295393719, 1.7788295719024707, 1.766..."
4,1,5,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.834012342776987, 1.8321873038112761, 1.8303...","[125, 130, 135, 140, 145, 150, 155, 160, 165, ...","[1.7909232748480652, 1.7893544285292378, 1.787...","[125, 170, 215, 260, 305, 350, 395, 440, 485, ...","[1.7909232748480652, 1.7773897124807985, 1.765..."
...,...,...,...,...,...,...,...,...
10681,47,10682,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.84724471159825, 1.845306403634897, 1.843378...","[1195, 1200, 1205, 1210, 1215, 1220, 1225, 123...","[1.4027216711179482, 1.3979441697822066, 1.393...","[1195, 1240, 1285, 1330, 1375, 1420, 1465, 151...","[1.4027216711179482, 1.3580654511370822, 1.309..."
10682,47,10683,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.84724471159825, 1.845306403634897, 1.843378...","[1200, 1205, 1210, 1215, 1220, 1225, 1230, 123...","[1.3979441697822066, 1.3931186077270967, 1.388...","[1200, 1245, 1290, 1335, 1380, 1425, 1470, 151...","[1.3979441697822066, 1.3528860367290538, 1.303..."
10683,47,10684,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.84724471159825, 1.845306403634897, 1.843378...","[1205, 1210, 1215, 1220, 1225, 1230, 1235, 124...","[1.3931186077270967, 1.3882458358804528, 1.383...","[1205, 1250, 1295, 1340, 1385, 1430, 1475, 152...","[1.3931186077270967, 1.3476662199521636, 1.298..."
10684,47,10685,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.84724471159825, 1.845306403634897, 1.843378...","[1210, 1215, 1220, 1225, 1230, 1235, 1240, 124...","[1.3882458358804528, 1.383326705170109, 1.3783...","[1210, 1255, 1300, 1345, 1390, 1435, 1480, 152...","[1.3882458358804528, 1.342406851734246, 1.2921..."


In [3]:
# Create a list to store aggregated data, preserving cell order
history_agg_list = []

# Assuming df is sorted by "Cell" as in the original order you want
# If not, sort it first: df = df.sort_values("Cell")

# Loop through all unique cell IDs in the order they appear in df
for cell_id in df["Cell"].unique():
    # Filter data for the current cell
    df_cell_i = df[df["Cell"] == cell_id]
    
    # Get the last sample for this cell
    last_sample = df_cell_i.iloc[-1]
    
    # Extract history and target arrays
    history_data = np.array(last_sample["History"])
    history_cycle = np.array(last_sample["History_Cycle"])
    target_data = np.array(last_sample["Target_expanded"])
    target_cycle = np.array(last_sample["Target_Cycle_Expanded"])
    
    # Concatenate the history and target arrays
    value_data = np.concatenate((history_data, target_data), axis=0)
    cycle_data = np.concatenate((history_cycle, target_cycle), axis=0)
    
    # Store the concatenated data for this cell
    history_agg_list.append({
        "Cell": cell_id,
        "Cycle": cycle_data,
        "Value": value_data
    })

# Convert to DataFrame to maintain order
history_agg_df = pd.DataFrame(history_agg_list)

In [4]:
history_agg_df

,Cell,Cycle,Value
0,1,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.834012342776987, 1.8321873038112761, 1.8303..."
1,2,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.845058495451667, 1.8419984665415141, 1.8389..."
2,3,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.840036111156223, 1.8372405290428224, 1.8344..."
3,4,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.836350335297539, 1.8345300178698336, 1.8327..."
4,5,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.827606331067727, 1.8254456233245018, 1.8232..."
5,6,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.832208014322532, 1.830601191788663, 1.82899..."
6,7,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.835825928297709, 1.8339510304090747, 1.8320..."
7,8,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.835988407303375, 1.833727801546082, 1.83148..."
8,9,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.831556955767155, 1.8301691290462887, 1.8287..."
9,10,"[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55,...","[1.836793169988596, 1.8351719721533677, 1.8335..."


In [ ]:
# Prepare the data
df_long = history_agg_df.explode(["Cycle", "Value"]).reset_index(drop=True)
df_long["Cycle"] = pd.to_numeric(df_long["Cycle"])
df_long["Value"] = pd.to_numeric(df_long["Value"])

# Create a single figure
fig = go.Figure()

# Get unique cell IDs
cell_ids = df_long["Cell"].unique()
colorscale = "Viridis"
num_cells = len(cell_ids)

# Add a trace for each cell with a color gradient
for i, cell_id in enumerate(cell_ids):
    df_cell = df_long[df_long["Cell"] == cell_id]
    color = sample_colorscale(colorscale, i / (num_cells - 1) if num_cells > 1 else 0)[0]
    fig.add_trace(
        go.Scatter(
            x=df_cell["Cycle"],
            y=df_cell["Value"],
            mode="lines",  # No markers for clarity with 47 lines
            line=dict(width=0.8, color=color),  # Thinner lines
            opacity=0.7,  # Slight transparency for overlap
            showlegend=False,  # No legend
        )
    )

# Determine the maximum cycle value to set x-axis range
max_cycle = df_long["Cycle"].max()
min_cycle = df_long["Cycle"].min()

# Customize layout for thesis quality
fig.update_layout(
    font=dict(family="Times New Roman", size=18),  # Match 11pt thesis body text
    title=dict(
        text="Aachen - Battery Capacity Over Cycles",
        x=0.5,
        xanchor="center",
        font=dict(size=24)  # Slightly smaller title
    ),
    xaxis_title="Cycle Number",
    yaxis_title="Capacity (Ah)",
    template="plotly_white",
    width=1102,  # 14 cm at 200 DPI
    height=709,  # 9 cm at 200 DPI
    margin=dict(l=60, r=60, t=80, b=60),
)

# Save as PDF for thesis
fig.write_image("reports/plots/capacity_degradation_aachen.pdf", engine="kaleido")

fig.show()

### Stanford

In [6]:
# Load the pickle file
batch1 = pickle.load(open(r'Testing_New_Dataset/batch1.pkl', 'rb'))
#remove batteries that do not reach 80% capacity
del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

bat_dict = {**batch1}

In [ ]:
# Assuming bat_dict is already loaded
# Define fixed capacity limits
min_cap = 0.5  # Minimum allowed discharge capacity
max_cap = 1.2  # Maximum allowed discharge capacity

# Create the figure
fig = go.Figure()

# Get battery keys and determine the number for color sampling
cell_keys = list(bat_dict.keys())
num_cells = len(cell_keys)
colorscale = "Viridis"

# Loop through each battery in bat_dict, clean the data, and assign a Viridis color
for i, cell_key in enumerate(cell_keys):
    cycle_numbers = np.array(bat_dict[cell_key]['summary']['cycle'])
    discharge_capacity = np.array(bat_dict[cell_key]['summary']['QD'])

    # Replace out-of-bound values with NaN
    discharge_capacity[(discharge_capacity < min_cap) | (discharge_capacity > max_cap)] = np.nan

    # Interpolate to fill NaN values (forward and backward filling)
    discharge_capacity = pd.Series(discharge_capacity)\
                          .interpolate(method='linear', limit_direction='both')\
                          .fillna(method='bfill')\
                          .fillna(method='ffill').values

    # Sample a color from the Viridis colorscale based on the battery's index
    color = px.colors.sample_colorscale(
        colorscale, 
        i / (num_cells - 1) if num_cells > 1 else 0
    )[0]

    fig.add_trace(go.Scatter(
        x=cycle_numbers,
        y=discharge_capacity,
        mode='lines',  # Display both lines and markers
        showlegend=False,      # Hide legend for clarity
        line=dict(width=0.8, color=color), # Assign Viridis color to the line
        opacity=0.7
    ))

# Update the layout for clarity and thesis-quality appearance
fig.update_layout(
    font=dict(family="Times New Roman", size=18),  # Use Times New Roman, 18pt for body text
    title=dict(
        text="Stanford - Battery Capacity Over Cycles",
        x=0.5,
        xanchor="center",
        font=dict(size=24)  # Title font size of 20pt
    ),
    xaxis_title="Cycle Number",
    yaxis_title="Discharge Capacity (Ah)",
    template="plotly_white",  # Clean white background template
    width=1102,  # 14 cm at 200 DPI
    height=709,  # 9 cm at 200 DPI
    margin=dict(l=60, r=60, t=80, b=60),
)

# Save as PDF for thesis
fig.write_image("reports/plots/capacity_degradation_stanford.pdf", engine="kaleido")

# Display the figure
fig.show()

/var/folders/z1/528lx89d6n3dbf9zzwd4qmw00000gn/T/ipykernel_27506/1262773560.py:25: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

/var/folders/z1/528lx89d6n3dbf9zzwd4qmw00000gn/T/ipykernel_27506/1262773560.py:26: FutureWarning:

Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

